In [0]:
CREATE OR REPLACE TABLE IDENTIFIER(:catalog || '.' || :schema || '.metadata') (
  path STRING,
  actual_spend_agreement_currency STRING, -- ENUM: 'CAD', 'EUR', 'USD', 'INR'
  actual_spend_usd STRING,
  affiliate_assignment STRING,
  agreement_currency STRING, -- ENUM: 'CAD', 'EUR', 'USD', 'INR'
  agreement_name STRING,
  agreement_title STRING,
  company_notice_contact_address STRING,
  company_notice_contact_email STRING,
  company_notice_contact_name STRING,
  contract_category STRING, -- ENUM: 'Construction & Maintenance Services', 'Corporate Communications', 'Corporate Services', 'Engineering & Design Services', 'Fabrication & Materials', 'Gases, Lubricants and Oils', 'Human Resources', 'Information Technology', 'MRO Supplies & Services', 'Process Automation', 'Professional Services', 'Property Services', 'Real Estate', 'Research and Technology', 'Responsible Care', 'Rotating Equipment', 'Travel', 'Water Treatment and Packaging'
  contract_owner STRING,
  counterparty_legal_entity_name STRING,
  counterparty_notice_contact_address STRING,
  counterparty_notice_contact_email STRING,
  counterparty_notice_contact_name STRING,
  counterparty_number STRING,
  does_corp_have_option_to_extend STRING,
  effective_date STRING,
  estimated_contract_value_agreement_currency STRING,
  estimated_contract_value_usd STRING,
  exchange_rate_to_usd STRING,
  expiry_date STRING,
  functional_unit STRING,
  is_change_of_control_clause STRING,
  is_non_solicitation_or_non_competition STRING,
  is_corp_guarantee STRING,
  is_corp_indemnity_third_party STRING,
  is_termination_for_convenience STRING,
  is_volume_rebate STRING,
  is_lease STRING,
  is_material_contract STRING,
  is_evergreen_agreement STRING,
  corp_legal_entity_name STRING,
  corp_site_location STRING,
  payment_terms STRING,
  project_number STRING,
  renewal_notice_period STRING,
  sap_outline_agreement_number STRING,
  third_party_assignment STRING,
  volume_rebate_payment_date STRING
)
USING DELTA;

MERGE INTO IDENTIFIER(:catalog || '.' || :schema || '.metadata') AS target
USING (
  SELECT
    path,
    metadata.*,
    metadata AS key_information
  FROM (
    SELECT
      path,
      from_json(AI_QUERY(
        "databricks-claude-sonnet-4-5",
        CONCAT(
          'You are a contract expert with the task of extracting key information.',

          :corporation_prompt,

          'Extract the following fields in json format from the vendor contract below. Use the exact field names and types as specified. For ENUM fields, only use the provided choices. If you are not confident about a field, return "".

          - actual_spend_agreement_currency: The actual spend in agreement currency. ENUM: CAD, EUR, USD, INR
          - actual_spend_usd: The actual spend in USD. Provide as a string, numbers only, no currency symbols.
          - affiliate_assignment: Affiliate assignment with/without consent; mutual/one-way (e.g., corp internal restructure). If not specified, return "".
          - agreement_currency: The currency of the contract. ENUM: CAD, EUR, USD, INR
          - agreement_name: The name of the agreement as stated in the contract.
          - agreement_title: The title of the agreement as stated in the contract.
          - company_notice_contact_address: The full address for company notice contact, including street, city, state/province, and country if available.
          - company_notice_contact_email: The email address for company notice contact.
          - company_notice_contact_name: The full name of the company notice contact person.
          - contract_category: The contract category. ENUM: Construction & Maintenance Services, Corporate Communications, Corporate Services, Engineering & Design Services, Fabrication & Materials, Gases, Lubricants and Oils, Human Resources, Information Technology, MRO Supplies & Services, Process Automation, Professional Services, Property Services, Real Estate, Research and Technology, Responsible Care, Rotating Equipment, Travel, Water Treatment and Packaging
          - contract_owner: The name or title of the contract owner within the corporation.
          - counterparty_legal_entity_name: The legal entity name of the counterparty as stated in the contract.
          - counterparty_notice_contact_address: The full address for counterparty notice contact, including street, city, state/province, and country if available.
          - counterparty_notice_contact_email: The email address for counterparty notice contact.
          - counterparty_notice_contact_name: The full name of the counterparty notice contact person.
          - counterparty_number: Counterparty number (e.g., SAP vendor number). Provide as a string.
          - does_corp_have_option_to_extend: Does the corporation have the option to extend the contract? Return "Yes", "No", or "" if not specified.
          - effective_date: The effective date of the contract in YYYY-MM-DD format if available.
          - estimated_contract_value_agreement_currency: Estimated contract value in agreement currency. Provide as a string, numbers only, no currency symbols.
          - estimated_contract_value_usd: Estimated contract value in USD. Provide as a string, numbers only, no currency symbols.
          - exchange_rate_to_usd: The exchange rate to USD as stated in the contract. Provide as a string.
          - expiry_date: The expiry date of the contract in YYYY-MM-DD format if available.
          - functional_unit: The functional unit or department responsible for the contract.
          - is_change_of_control_clause: Is there a change of control clause? Return "Yes", "No", or "" if not specified.
          - is_non_solicitation_or_non_competition: Is there a non-solicitation or non-competition agreement? Return "Yes", "No", or "" if not specified.
          - is_corp_guarantee: Is there a corporate guarantee? Return "Yes", "No", or "" if not specified.
          - is_corp_indemnity_third_party: Is there a corporate indemnity of a third party? Return "Yes", "No", or "" if not specified.
          - is_termination_for_convenience: Is there a termination for convenience clause? Return "Yes", "No", or "" if not specified.
          - is_volume_rebate: Is there a volume rebate? Return "Yes", "No", or "" if not specified.
          - is_lease: Is this contract a lease? Return "Yes", "No", or "" if not specified.
          - is_material_contract: Is this a material contract? Return "Yes", "No", or "" if not specified.
          - is_evergreen_agreement: Is this an evergreen agreement? Return "Yes", "No", or "" if not specified.
          - corp_legal_entity_name: The legal entity name of the corporation as stated in the contract.
          - corp_site_location: The site location of the corporation as stated in the contract.
          - payment_terms: The payment terms as stated in the contract (e.g., "Net 30 days").
          - project_number: The project number associated with the contract, if available.
          - renewal_notice_period: The renewal notice period as stated in the contract (e.g., "60 days").
          - sap_outline_agreement_number: The SAP outline agreement number, if available.
          - third_party_assignment: Third party assignment with/without consent; mutual/one-way (e.g., sale of asset). If not specified, return "".
          - volume_rebate_payment_date: The payment date for volume rebate, if specified, in YYYY-MM-DD format.

          Think carefully about the extraction and review your work.
          For ENUM fields, only use the provided choices. 
          If you are not confident about a field, return "".
          Return the output in json format. 

          CONTRACT INFORMATION',
          'Vendor Name:', vendor_name, '\n',
          'File Name:', file_name, '\n',
          'Text:', truncated, '\n'
        ),
        responseFormat => 'STRUCT<result:STRUCT<
          actual_spend_agreement_currency:STRING,
          actual_spend_usd:STRING,
          affiliate_assignment:STRING,
          agreement_currency:STRING,
          agreement_name:STRING,
          agreement_title:STRING,
          company_notice_contact_address:STRING,
          company_notice_contact_email:STRING,
          company_notice_contact_name:STRING,
          contract_category:STRING,
          contract_owner:STRING,
          counterparty_legal_entity_name:STRING,
          counterparty_notice_contact_address:STRING,
          counterparty_notice_contact_email:STRING,
          counterparty_notice_contact_name:STRING,
          counterparty_number:STRING,
          does_corp_have_option_to_extend:STRING,
          effective_date:STRING,
          estimated_contract_value_agreement_currency:STRING,
          estimated_contract_value_usd:STRING,
          exchange_rate_to_usd:STRING,
          expiry_date:STRING,
          functional_unit:STRING,
          is_change_of_control_clause:STRING,
          is_non_solicitation_or_non_competition:STRING,
          is_corp_guarantee:STRING,
          is_corp_indemnity_third_party:STRING,
          is_termination_for_convenience:STRING,
          is_volume_rebate:STRING,
          is_lease:STRING,
          is_material_contract:STRING,
          is_evergreen_agreement:STRING,
          corp_legal_entity_name:STRING,
          corp_site_location:STRING,
          payment_terms:STRING,
          project_number:STRING,
          renewal_notice_period:STRING,
          sap_outline_agreement_number:STRING,
          third_party_assignment:STRING,
          volume_rebate_payment_date:STRING
        >>'
      ),
      'STRUCT<
          actual_spend_agreement_currency:STRING,
          actual_spend_usd:STRING,
          affiliate_assignment:STRING,
          agreement_currency:STRING,
          agreement_name:STRING,
          agreement_title:STRING,
          company_notice_contact_address:STRING,
          company_notice_contact_email:STRING,
          company_notice_contact_name:STRING,
          contract_category:STRING,
          contract_owner:STRING,
          counterparty_legal_entity_name:STRING,
          counterparty_notice_contact_address:STRING,
          counterparty_notice_contact_email:STRING,
          counterparty_notice_contact_name:STRING,
          counterparty_number:STRING,
          does_corp_have_option_to_extend:STRING,
          effective_date:STRING,
          estimated_contract_value_agreement_currency:STRING,
          estimated_contract_value_usd:STRING,
          exchange_rate_to_usd:STRING,
          expiry_date:STRING,
          functional_unit:STRING,
          is_change_of_control_clause:STRING,
          is_non_solicitation_or_non_competition:STRING,
          is_corp_guarantee:STRING,
          is_corp_indemnity_third_party:STRING,
          is_termination_for_convenience:STRING,
          is_volume_rebate:STRING,
          is_lease:STRING,
          is_material_contract:STRING,
          is_evergreen_agreement:STRING,
          corp_legal_entity_name:STRING,
          corp_site_location:STRING,
          payment_terms:STRING,
          project_number:STRING,
          renewal_notice_period:STRING,
          sap_outline_agreement_number:STRING,
          third_party_assignment:STRING,
          volume_rebate_payment_date:STRING
        >'
      )
      as metadata
    FROM IDENTIFIER(:catalog || '.' || :schema || '.flat')
  )
) AS source
ON target.path = source.path
WHEN NOT MATCHED THEN INSERT *;